
## Importing libraries

In [ ]:
import tensorflow as tf
from tqdm import tqdm
import numpy as np
import os
import IPython.display as display
from PIL import Image
import pandas as pd
import shutil
import matplotlib.pyplot as plt

## Path Initialization

In [ ]:
TRAIN_TFRECORD_PATH = '/kaggle/input/cassava-leaf-disease-classification/train_tfrecords/' # Specify path for tfrecord files
TRAIN_IMAGES_FROM_TFRECORD_PATH = '/kaggle/working/Train_Images/'                     # Specify path where you want to extract images from tfrecord

In [ ]:
# Let's create required path for TRAIN images

if os.path.exists(TRAIN_IMAGES_FROM_TFRECORD_PATH):
    print('Looks like already exists')
else:
    os.makedirs(TRAIN_IMAGES_FROM_TFRECORD_PATH)
    print(f"Great , You have created required folder at {TRAIN_IMAGES_FROM_TFRECORD_PATH}")

## Let's make hand dirty by converting tfrecord to jpg

In [ ]:
# # Create a dictionary describing the features. This 
image_feature_description = {
    'image': tf.io.FixedLenFeature([], tf.string),  
    'image_name': tf.io.FixedLenFeature([], tf.string),
    'target': tf.io.FixedLenFeature([], tf.int64),
}

def _parse_image_function(example_proto):
  # Parse the input tf.train.Example proto using the dictionary above.
  return tf.io.parse_single_example(example_proto, image_feature_description)

In [ ]:
# I want to store tfrecord filename in variable, filename
filename = [TRAIN_TFRECORD_PATH + i for i in os.listdir(TRAIN_TFRECORD_PATH)]
filename[:2]

- Code without using multiprocessing take appox = 50 iter/s, We have around 21,397 images which will cost us 21397/50 = 427 sec =7.13 minutes,

In [ ]:

df_to_store_imagename_and_info = pd.DataFrame()
image_name_to_df = []
target_name_to_df = []

# Here i am converting only first two tfrecord file, so if you want to convert all tfrecord files, then remove square bracket
# from parsed parameter to the TFRecordDataset(filename = filename)

raw_image_dataset = tf.data.TFRecordDataset(filenames=filename[:2]) # parsing tfrecords path to the TFrecordDataset function from tensorflow
parsed_image_dataset = raw_image_dataset.map(_parse_image_function) # Basically our tfrecorf file is in the shape, where it contains
                                                              # dict with (array of image, image_name, target variable)


for image_features in tqdm(parsed_image_dataset):
    image_raw = image_features['image'].numpy()        # accessing image array  
    # display.display(display.Image(data=image_raw))
    array = tf.io.decode_image(image_raw, dtype=tf.dtypes.uint8, expand_animations=True).numpy() # Decoding image
    im = Image.fromarray(array)                        # converting as a image
    image_path = TRAIN_IMAGES_FROM_TFRECORD_PATH +str(image_features['image_name'].numpy(), 'utf8')  # specifying image path with image name
    im.save(image_path)
    
    # we are storing image path and target variable for later use 
    image_name_to_df.append(str(image_features['image_name'].numpy(), 'utf8'))
    target_name_to_df.append(image_features['target'].numpy())

df_to_store_imagename_and_info['Image_name'] = image_name_to_df
df_to_store_imagename_and_info['Target'] = target_name_to_df

print('Great you have converted from tfrecord to jpg format :)')

In [ ]:
df_to_store_imagename_and_info.head(3)

In [ ]:
# we have extracted tfrecrd to jpg format, We have extracted 2676 out of 21397 images, Coooool
len(os.listdir(TRAIN_IMAGES_FROM_TFRECORD_PATH))

## Visualizing image

In [ ]:
img = tf.io.read_file(TRAIN_IMAGES_FROM_TFRECORD_PATH + os.listdir(TRAIN_IMAGES_FROM_TFRECORD_PATH)[0] )
img = tf.image.decode_image(img).numpy()
plt.imshow(img)

In [ ]:
# I am removing the extracted files from output directory , Please comment this line so that you will have images in output directory
shutil.rmtree(TRAIN_IMAGES_FROM_TFRECORD_PATH)

In [ ]:
#Congratulations,  Finally you have reached end of this notebook, If you found useful do like and comment, If I can improve this in anyway
# Please let me know :)
